In [1]:
from main import transform, load_to_mongodb

Scraping from page: 1: https://www.vatanbilgisayar.com/bilgisayar/?page=1
Scraping from page: 2: https://www.vatanbilgisayar.com/bilgisayar/?page=2
Scraping from page: 3: https://www.vatanbilgisayar.com/bilgisayar/?page=3
Scraping from page: 4: https://www.vatanbilgisayar.com/bilgisayar/?page=4
Scraping from page: 5: https://www.vatanbilgisayar.com/bilgisayar/?page=5
Scraping from page: 6: https://www.vatanbilgisayar.com/bilgisayar/?page=6
Scraping from page: 7: https://www.vatanbilgisayar.com/bilgisayar/?page=7
Scraping from page: 8: https://www.vatanbilgisayar.com/bilgisayar/?page=8
Scraping from page: 9: https://www.vatanbilgisayar.com/bilgisayar/?page=9
Scraping from page: 10: https://www.vatanbilgisayar.com/bilgisayar/?page=10
Scraping from page: 11: https://www.vatanbilgisayar.com/bilgisayar/?page=11
Scraping from page: 12: https://www.vatanbilgisayar.com/bilgisayar/?page=12
Scraping from page: 13: https://www.vatanbilgisayar.com/bilgisayar/?page=13
Scraping from page: 14: https:

In [2]:
URL = 'https://www.vatanbilgisayar.com/bilgisayar/?page='
collection, client = load_to_mongodb(transform)

Data successfully saved to MongoDB.


In [3]:
collection.create_index([('BRAND', 1)]) 

'BRAND_1'

In [4]:
collection.create_index([('NAME', 1)]) 

'NAME_1'

In [5]:
results_of_avg_computer_prices = collection.aggregate([
    {
        '$group': {
            '_id': '$BRAND',
            'average_price_try': {'$avg': '$PRICE_TRY'},
            'average_price_usd': {'$avg': '$PRICE_USD'},
            'average_price_euro': {'$avg': '$PRICE_EURO'}
        }
    },
    {
        '$sort': {'average_price_try': -1}
    },
    {'$limit':10}
])

print("Top 10 Brands with the Highest Average Computer Prices:")
for result in results_of_avg_computer_prices:
    print(f"Brand: {result['_id']}, {result['average_price_try']:.0f}₺, {result['average_price_usd']:.0f}$, {result['average_price_euro']:.0f}€")

Top 10 Brands with the Highest Average Computer Prices:
Brand: GIGABYTE, 114293₺, 3337$, 3094€
Brand: DELL, 91626₺, 2675$, 2480€
Brand: MSI, 71517₺, 2088$, 1936€
Brand: MACBOOK, 68005₺, 1985$, 1840€
Brand: ASUS, 61353₺, 1791$, 1660€
Brand: APPLE, 59124₺, 1726$, 1600€
Brand: LENOVO, 53473₺, 1561$, 1447€
Brand: HP, 51226₺, 1495$, 1386€
Brand: IPAD, 44415₺, 1296$, 1202€
Brand: EXPER, 34501₺, 1007$, 934€


In [6]:
results_of_avg_computer_prices_reverse = collection.aggregate([
    {'$group': {
        '_id': '$BRAND',
        'average_price_try': {'$avg': '$PRICE_TRY'},
        'average_price_usd': {'$avg': '$PRICE_USD'},
        'average_price_euro': {'$avg': '$PRICE_EURO'}
    }
    },
        {
            '$sort': {'average_price_try': 1}
        },
    {
        '$limit':10
    }
])

print("Top 10 Brands with the Lowest Average Computer Prices:")
for result in results_of_avg_computer_prices_reverse:
    print(f"Brand: {result['_id']}, {result['average_price_try']:.0f}₺, {result['average_price_usd']:.0f}$S, {result['average_price_euro']:.0f}€")

Top 10 Brands with the Lowest Average Computer Prices:
Brand: S-LINK, 413₺, 12$S, 11€
Brand: T720, 711₺, 20$S, 19€
Brand: EVERPAD, 3982₺, 116$S, 107€
Brand: HOMETECH, 5999₺, 175$S, 162€
Brand: TCL, 7099₺, 207$S, 192€
Brand: HONOR, 10528₺, 307$S, 284€
Brand: REDMI, 12799₺, 373$S, 346€
Brand: AIDATA, 13332₺, 389$S, 360€
Brand: SAMSUNG, 17829₺, 520$S, 482€
Brand: WARP, 25146₺, 734$S, 680€


In [7]:
results_of_avg_review_counts = collection.aggregate([
    {
        '$group': {
            '_id': '$BRAND',
            'average_review_count': {'$avg': '$REVIEWS'} 
        }
    },
    {
        '$sort': {'average_review_count': -1}
    },
    {
        '$limit': 10
    },
    {
        '$project': {
            '_id': 1,
            'average_review_count': {'$round': ['$average_review_count', 0]}
        }
    }
])

print('Top 10 Brands with the Highest Average Review Count:')
for result in results_of_avg_review_counts:
    print(f"Brand: {result['_id']}, Average_Reviews: {result['average_review_count']}")

Top 10 Brands with the Highest Average Review Count:
Brand: SAMSUNG, Average_Reviews: 44.0
Brand: MACBOOK, Average_Reviews: 24.0
Brand: IPAD, Average_Reviews: 23.0
Brand: HONOR, Average_Reviews: 19.0
Brand: EVERPAD, Average_Reviews: 13.0
Brand: HOMETECH, Average_Reviews: 12.0
Brand: ACER, Average_Reviews: 10.0
Brand: HUAWEI, Average_Reviews: 7.0
Brand: LENOVO, Average_Reviews: 6.0
Brand: REDMI, Average_Reviews: 5.0


In [8]:
count_of_products_by_brand = list(collection.aggregate([
    {'$group':
         {'_id': '$BRAND',
          'count_of_products': {'$sum': 1}}},
    {
        '$sort': {'count_of_products': -1}
    },
    {
        '$limit': 15
    }
]))

share = sum(result['count_of_products'] for result in count_of_products_by_brand)


print("Top 15 Computers by Brand and Their Market Shares")
for result in count_of_products_by_brand:
    market_share = result['count_of_products'] / share * 100
    print(f"Brand: {result['_id']}, Count of Products: {result['count_of_products']}, Market Share: {market_share:.2f}%")    

Top 15 Computers by Brand and Their Market Shares
Brand: IPAD, Count of Products: 960, Market Share: 26.19%
Brand: LENOVO, Count of Products: 488, Market Share: 13.31%
Brand: MACBOOK, Count of Products: 416, Market Share: 11.35%
Brand: ASUS, Count of Products: 360, Market Share: 9.82%
Brand: SAMSUNG, Count of Products: 264, Market Share: 7.20%
Brand: CASPER, Count of Products: 248, Market Share: 6.76%
Brand: ACER, Count of Products: 184, Market Share: 5.02%
Brand: MSI, Count of Products: 176, Market Share: 4.80%
Brand: HP, Count of Products: 128, Market Share: 3.49%
Brand: HUAWEI, Count of Products: 120, Market Share: 3.27%
Brand: WARP, Count of Products: 114, Market Share: 3.11%
Brand: APPLE, Count of Products: 64, Market Share: 1.75%
Brand: HONOR, Count of Products: 56, Market Share: 1.53%
Brand: EXPER, Count of Products: 48, Market Share: 1.31%
Brand: DELL, Count of Products: 40, Market Share: 1.09%


In [9]:
max_price_product_by_brand = collection.aggregate([
    {'$project': 
        {
        '_id': '$BRAND',
        'NAME': '$NAME',
        'REVIEWS': '$REVIEWS',
        'max_prices_TRY': {'$max': '$PRICE_TRY'},
        'max_prices_USD': {'$max': '$PRICE_USD'},
        'max_prices_EURO': {'$max': '$PRICE_EURO'}
    }
    },
    {
        '$sort': {'max_prices_TRY': -1}
    },
    {
        '$limit': 1
    },
])

print("The Most Expensive Computer:")
for result in max_price_product_by_brand:
    print(f"-->Brand: {result['_id']}\n"
          f"-->Product: {result['NAME'].title()}\n"
          f"-->Reviews: {result['REVIEWS']}\n"
          f"-->{result['max_prices_TRY']}₺\n"
          f"-->{result['max_prices_USD']}$\n"
          f"-->{result['max_prices_EURO']}€")

The Most Expensive Computer:
-->Brand: MSI
-->Product: Msi Titan 18 Hx 14.Nesil Core I9 14900Hx-Rtx4090 16Gb-128Gb-4Tb Ssd-18Inc-W11Pro
-->Reviews: 0
-->239800₺
-->7001$
-->6491€


In [10]:
max_reviews_product_by_brand = collection.aggregate([
    {'$project':
         {
             '_id': '$BRAND',
             'NAME': '$NAME',
             'count_of_reviews': '$REVIEWS',
             'PRICE_TRY': '$PRICE_TRY', 
             'PRICE_USD': '$PRICE_USD',
             'PRICE_EURO': '$PRICE_EURO'
         }
    },
    {
        '$sort': {'count_of_reviews': -1}
    },
    {
        '$limit': 1
    }
])

print("The Most Reviewed Product by Brand:")
for result in max_reviews_product_by_brand:
    print(f"-->Brand: {result['_id']}\n"
          f"-->Product: {result['NAME'].title()}\n"
          f"-->Reviews: {result['count_of_reviews']}\n"
          f"-->{result['PRICE_TRY']:.0f}₺\n"
          f"-->{result['PRICE_USD']:.0f}$\n"
          f"-->{result['PRICE_EURO']:.0f}€")

The Most Reviewed Product by Brand:
-->Brand: IPAD
-->Product: Ipad 9.Nesil 64Gb Wi-Fi 10.2''Retina Ekran Ipados Tablet Space Gray
-->Reviews: 294
-->12999₺
-->379$
-->351€


In [11]:
def input_prices():
    active = 1
    while active != 0:
        try:
            min_price = int(input("Enter the min price: "))
            max_price = int(input("Enter the max price: "))
            active = 0
        except ValueError:
            print("Please enter a number\n")  
    return min_price, max_price

def input_sort() -> str:
    active = 1
    while active != 0:
        try:
            sort = str(input("Enter the sort: ASCENDING (ASC) or DESCENDING (DESC)"))
            if sort.upper() == 'ASC':
                active = 0
                sort = 1
            elif sort.upper() == 'DESC':
                active = 0
                sort = -1
            else:
                print("Please enter either ASC or DESC")
        except TypeError:
            print("Please enter either ASC or DESC")
    return sort

min_price, max_price = input_prices()
sort = input_sort()

filter_query = collection.aggregate([
    {'$project':
         {
             '_id': '$BRAND',
             'Product': '$NAME',
             'PRICE_TRY': '$PRICE_TRY',
             'REVIEWS': '$REVIEWS'
         }
    },
    {
        '$sort': {'PRICE_TRY': sort}
    },
    {
        '$match': {
            'PRICE_TRY': {'$gte': min_price, '$lte': max_price}
        }
    }  
])

sort = 'Descending' if sort == -1 else 'Ascending'
print(f"Results filtered between {min_price} and {max_price}, sorted by {sort}")
for result in filter_query:
    print(f"{result['_id']} - {result['Product']}: {result['PRICE_TRY']:.0f}₺ ({result['REVIEWS']})")

Results filtered between 40000 and 50000, sorted by Descending
MACBOOK - macbook air mly23tu/a m2 8gb-512gb ssd-liquid retina-13.6inc-yıldız ışığı: 49999₺ (78)
MACBOOK - macbook air mlxx3tu/a m2 8gb-512gb ssd-liquid retina-13.6inc-uzay grisi: 49999₺ (78)
ACER - acer nitro 5 12.nesil core i7 12650h-rtx4060 8gb-16gb-1tb ssd-15.6inc-w11: 49999₺ (6)
MACBOOK - macbook air mly43tu/a m2 8gb-512gb ssd-liquid retina-13.6inc-gece yarısı: 49999₺ (78)
LENOVO - lenovo yoga 7 core ultra 5 125u-16gb-512gb ssd-14inc-w11: 49999₺ (0)
MACBOOK - macbook air mly43tu/a m2 8gb-512gb ssd-liquid retina-13.6inc-gece yarısı: 49999₺ (78)
MACBOOK - macbook air mlxx3tu/a m2 8gb-512gb ssd-liquid retina-13.6inc-uzay grisi: 49999₺ (78)
MACBOOK - macbook air mly03tu/a m2 8gb-512gb ssd-liquid retina-13.6inc-gümüş: 49999₺ (78)
ACER - acer nitro 5 12.nesil core i7 12650h-rtx4060 8gb-16gb-1tb ssd-15.6inc-w11: 49999₺ (6)
MACBOOK - macbook air mly43tu/a m2 8gb-512gb ssd-liquid retina-13.6inc-gece yarısı: 49999₺ (78)
MACBOOK 

In [12]:
client.close()